In [3]:
import pandas as pd
import sys
import shutil
import tqdm

In [16]:
df = pd.read_csv("jersey.csv")

In [17]:

df = df.drop(columns=["num"])
df.head()

,filename,team,jersey_number
0,Chelsea__Bournemouth/Chelsea__Bournemouth_1_0.png,0,5
1,Chelsea__Bournemouth/Chelsea__Bournemouth_2_0.png,0,5
2,Chelsea__Bournemouth/Chelsea__Bournemouth_3_0.png,0,5
3,Chelsea__Bournemouth/Chelsea__Bournemouth_4_0.png,0,5
4,Chelsea__Bournemouth/Chelsea__Bournemouth_5_0.png,0,5


In [18]:
filenames = df["filename"].tolist()

In [19]:
for file in tqdm.tqdm(filenames):
    try:
        shutil.copy(f"ml_project_data/{file}", f"ml_project_data/dataset/{file.split('/')[-1]}")
    except Exception as e:
        print(e)
        print(file)

100%|██████████| 5101/5101 [00:04<00:00, 1034.28it/s]


In [1]:
from PIL import Image
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
from torchvision.io import read_image
import pandas as pd
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import tqdm
import torch.nn as nn
import torch.optim as optim
import cv2
import random

In [2]:
class FootballCNN(nn.Module):
    def __init__(self, num_classes=19):
        super(FootballCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
            # result after conv2d is 16x64x64
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            #result after maxpool is 16x32x32

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            # 32x32x32
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 32x16x16


            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            # 64x16x16
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 64x8x8

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            #128x828
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
            #128x4x4
         )
        
        self.fc_layers = nn.Sequential(
            nn.Dropout(0.25),
            nn.Linear(128 * 4 * 4, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, num_classes),
            # nn.Softmax(dim=1),
        )


    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device("cpu")
model = FootballCNN(num_classes=100).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [20]:

model.load_state_dict(torch.load("/Users/shevdan/Documents/Programming/ML/Player_recognition/models/jersey_model_1.pt"))
model.eval()

FootballCNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, 

In [4]:
device = torch.device("cpu")
model_team = FootballCNN(num_classes=2).to(device)


In [5]:

model_team.load_state_dict(torch.load("/Users/shevdan/Documents/Programming/ML/Player_recognition/models/team_model.pt"))
model_team.eval()

FootballCNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, 

In [6]:
def get_prediction(img_filename, model):
    image = cv2.imread(img_filename)
    if len(image.shape) > 2 and image.shape[2] == 4:
    #convert the image from RGBA2RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGB)
    image = np.array(image)
    print(image.shape)
    transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    ])
    image = transform(image)
    # print(image.shape)
    image = image.reshape(-1, image.shape[0], image.shape[1], image.shape[2] )
    
    outputs = model(image)
    print(outputs)
    max_vals, predicted = torch.max(outputs.data, 1)
    print(max_vals, predicted)
    return predicted.item()


In [7]:
def get_pred(img_arr, model):
    transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    ])
    image = transform(img_arr)
    
    image = image.reshape(-1, image.shape[0], image.shape[1], image.shape[2] )
    
    outputs = model(image)
    max_vals, predicted = torch.max(outputs.data, 1)
    return max_vals.item(), predicted.item()

In [58]:
def plot_one_box(x, img, color=None, label=None, line_thickness=1):

    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA) 
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)


In [24]:
import json

In [57]:
def classify_and_plot(img_path: str, xywh: list, model_jersey, model_team, data_dct):
    team_corr = {"0": "Chelsea", "1": "Man City", "": ""}
    img = cv2.imread(img_path)
    imH, imW, _ = img.shape
    
    x_center, y_center, box_w, box_h, = xywh[0], xywh[1], xywh[2], xywh[3]
    x_l, x_r = max(x_center - box_w // 2, 0), max(x_center + box_w //2, 0)
    y_u, y_d = max(y_center + box_h // 2, 0), max(y_center - box_h // 2, 0)
    if data_dct["to_classify"]:
        pil_img = Image.open(img_path)
        pil_img = pil_img.crop((x_l, y_d, x_r, y_u))
        jersey_conf, jersey_pred = get_pred(np.array(pil_img), model_jersey)
        team_conf, team_pred = get_pred(np.array(pil_img), model_team)
        # print(f"{img_path}, JERSEY: {jersey_conf}; {jersey_pred}, TEAM: {team_conf}, {team_pred}")
        if jersey_conf > 5:
            data_dct["to_classify"] = False
            data_dct["team"] = str(team_pred)
            data_dct["jersey"] = str(jersey_pred)
    # print(data_dct)
    with open("/Users/shevdan/Documents/Programming/ML/Player_recognition/results/Premier League_stats_2023-03-02.json") as jsonfile:
            names = json.load(jsonfile)
    # names[team_corr[str(data_dct['team'])]][str(data_dct['jersey'])]
    label = "" if data_dct["to_classify"] else f"Team: {team_corr[str(data_dct['team'])]}. Name: {names[team_corr[str(data_dct['team'])]][str(data_dct['jersey'])]}. Stat: {data_dct['stats']}"
    # print(label)
    plot_one_box([x_l, y_d, x_r, y_u], img, color=(255, 0, 0), label=label)
    filename, ext = os.path.splitext(img_path.split('/')[-1])
    bbx_dir_path = f"{'/'.join(img_path.split('/')[:-2])}/{img_path.split('/')[-2]}_bbx"
    if not os.path.exists(bbx_dir_path):
        os.mkdir(bbx_dir_path)
    cv2.imwrite(f"{bbx_dir_path}/{filename}_bbox{ext}", img)
    return data_dct

    
    
    

        


In [47]:
data = {
    "to_classify": False,
    "team": "",
    "jersey": "",
    "stats": "Some stats"
}
classify_and_plot("/Users/shevdan/Documents/Programming/ML/Player_recognition/images/img2.png", [0.388468, 0.882694, 0.0300732, 0.162967], model, model_team, data)

(1428, 1910, 1542, 1588)


In [33]:
vid_num = 2
vid_folder_path = f"/Users/shevdan/Documents/Programming/ML/Player_recognition/test_data/img/{vid_num}"
text_folders = f"/Users/shevdan/Documents/Programming/ML/Player_recognition/yolov7/runs/detect_{vid_num}/exp"
img = cv2.imread(f"{vid_folder_path}/1.png")
imH, imW, _ = img.shape

In [16]:
import random

In [ ]:
data = {
    "to_classify": True,
    "team": "",
    "jersey": "",
    "stats": "Some stats"
}
for i in range(len(os.listdir(vid_folder_path))):
    bdnbxes = []
    boxes_txt = f"{text_folders}/labels/{i+1}.txt" if i == 0 else f"{text_folders}{i+1}/labels/{i+1}.txt"
    with open(boxes_txt, mode="r") as labels:
        for line in labels:
            box = line.strip().split()
            if box[0] != "0":
                continue
            xywh = [float(coord) for coord in box[1:]]
            bdnbxes.append( [int(xywh[0] * imW), int(xywh[1] * imH), int(xywh[2] * imW), int(xywh[3] * imH)])
    
    if i == 0:
        # chosen_dude = random.choice(bdnbxes)
        chosen_dude = bdnbxes[7]
    else:
        bdnbxes.sort(key=lambda x: ((x[0] - chosen_dude[0]) ** 2 + (x[1] - chosen_dude[1])**2))
        chosen_dude = bdnbxes[0]
            
    data = classify_and_plot(f"{vid_folder_path}/{i + 1}.png", chosen_dude, model, model_team, data)

